# Notebook para el despliegue del modelo en la nube de Azure

---

In [1]:
import json
import os
from dotenv import load_dotenv
load_dotenv()

# Usamos una variable de entorno oculta para cargar el id de la suscripción, se trata de un archivo .env que debe estar en el mismo directorio que este script
my_secret = os.getenv("MY_SECRET")
tenant_id = os.getenv("TENANT_ID")

my_id = my_secret

from azureml.core.authentication import InteractiveLoginAuthentication
ia = InteractiveLoginAuthentication(tenant_id=tenant_id)

from azureml.core import Workspace

ws = Workspace.create(name="workspace7",
                      subscription_id = my_id,
                      resource_group = "clasecloud8",
                      location = "eastus2",)

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="linear_model.pkl",
                                  model_name=mname,
                                  workspace=ws)


KeyboardInterrupt: 

In [ ]:
################ score.py ###################


scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model
from datetime import date, timedelta

def init():
  global model
  model_path = Model.get_model_path('model')
  model = joblib.load(model_path)

def prepare_data(df:pd.DataFrame) -> pd.DataFrame:
    df = df[['FirstName', 'ModifiedDate']].sort_values(by='ModifiedDate')
    df['FirstNameLen'] = df['FirstName'].str.len()
    df["ModifiedDate"] = pd.to_datetime(df["ModifiedDate"]) - pd.to_datetime(df["ModifiedDate"]).min()
    df["ModifiedDate"] = df["ModifiedDate"].dt.days
    
    return df[['FirstNameLen', 'ModifiedDate']].reset_index(drop=True)

def int_to_date(day_int: int, start_date: date = date(2005, 7, 1)) -> date:
    
    result=start_date + timedelta(days=day_int)
    result = result.strftime("%Y-%m-%d")
    return result

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    data = prepare_data(data)

    columns_model = ["FirstNameLen"]

    data_dummies = data[columns_model]

    result = model.predict(data_dummies).tolist()

    for i in range(len(result)):
      result[i] = int_to_date(result[i])
   

    return json.dumps(result)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()


In [ ]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-dates")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) ## Usar los paquetes correctos


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) 

service = Model.deploy(workspace=ws,
                       name='service-2',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\angel\AppData\Local\Temp\ipykernel_21448\4056085815.py:16: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [ ]:
#Publicar el servicio y guardar el URI en un archivo json
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri2.json", "w")
file.write(scoreuri)
file.close()


Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: ce551e91-4375-4b0b-821a-4994f66e0f48
More information can be found using '.get_logs()'
Error:
{
  "code": "ContainerGroupQuotaReached",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: Resource type 'Microsoft.ContainerInstance/containerGroups' container group quota 'StandardCores' exceeded in region 'westus'. Limit: '6', Usage: '4.2' Requested: '2.1'.."
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: ce551e91-4375-4b0b-821a-4994f66e0f48
More information can be found using '.get_logs()'
Error:
{
  "code": "ContainerGroupQuotaReached",
  "statusCode": 400,
  "message": "ACI Service request failed. Reason: Resource type 'Microsoft.ContainerInstance/containerGroups' container group quota 'StandardCores' exceeded in region 'westus'. Limit: '6', Usage: '4.2' Requested: '2.1'.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: ce551e91-4375-4b0b-821a-4994f66e0f48\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"ContainerGroupQuotaReached\",\n  \"statusCode\": 400,\n  \"message\": \"ACI Service request failed. Reason: Resource type 'Microsoft.ContainerInstance/c